In [1]:
from talib import MA,STOCH
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("SPY.csv",date_parser=True)

In [3]:
df_train = df[ df["Date"]<'2017-01-01' ].copy()
# df_validation = df[(df['Date'] >= '2017-01-01') & (df['Date'] < "2018-01-01")].copy()
df_validation = df[df["Date"] >= "2017-01-01"].copy()
df_plot = df[df["Date"] >= "2018-01-01"].copy()

# Graph Plotting

# Add Technical Analysis

In [4]:
ma_10_train = MA(df_train["Close"], timeperiod=10, matype=0)
ma_30_train = MA(df_train["Close"], timeperiod=30, matype=0)
k_train, d_train = STOCH(df_train["High"], df_train["Low"], df_train["Close"],fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_train["MA10"] = ma_10_train
df_train["MA30"] = ma_30_train
df_train["K"] = k_train
df_train["D"] = d_train

ma_10_validation = MA(df_validation["Close"], timeperiod=10, matype=0)
ma_30_validation = MA(df_validation["Close"], timeperiod=30, matype=0)
k_validation, d_validation = STOCH(df_validation["High"], df_validation["Low"], df_validation["Close"],fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_validation["MA10"] = ma_10_validation
df_validation["MA30"] = ma_30_validation
df_validation["K"] = k_validation
df_validation["D"] = d_validation

In [5]:
df_validation.shape

(501, 11)

## Drop dates and NA

In [6]:
df_train = df_train.dropna(axis=0)
df_train = df_train.drop(["Date"], axis=1)

df_validation = df_validation.dropna(axis=0)
df_validation = df_validation.drop(["Date"], axis=1)

In [7]:
df_validation.shape

(472, 10)

## Normalize data

In [8]:
def normalizeDataframe(data_frame):
    normalize_df = data_frame.copy()
    for column in normalize_df.columns:
        min_value = min(normalize_df[column])
        max_value = max(normalize_df[column])
        normalize_df[column] = (normalize_df[column] - min_value) / (max_value - min_value)
    return normalize_df

In [9]:
df_train = normalizeDataframe(df_train)
df_validation = normalizeDataframe(df_validation)

In [10]:
df_validation.shape

(472, 10)

## Prepare X, y train and validation for RNN

In [11]:
data_train = df_train.values
data_validation = df_validation.values

In [12]:
X_train = []
y_train = []
X_validation = []
y_validation = []

In [13]:
for i in range(30,data_train.shape[0]):
    X_train.append(data_train[i-30:i])
    y_train.append(data_train[i, 0])

for i in range(30, data_validation.shape[0]):
    X_validation.append(data_validation[i-30:i])
    y_validation.append(data_validation[i,0])

In [14]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_validation, y_validation = np.array(X_validation), np.array(y_validation)

In [15]:
X_train.shape
# y_train.shape

(5734, 30, 10)

# Building Models

In [35]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, SimpleRNN, GRU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [36]:
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=8)

## Vanilla RNN

In [37]:
regressor_RNN = Sequential()
regressor_RNN.add(SimpleRNN(units = 32, activation = 'tanh', input_shape = (X_train.shape[1], X_train.shape[2])))
regressor_RNN.add(Dense(units = 1))

In [38]:
regressor_RNN.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 32)                1376      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________


In [39]:
checkpoint_RNN = ModelCheckpoint(filepath="best_params_RNN.hdf5", monitor="val_loss",verbose=1,save_best_only=True)

In [40]:
regressor_RNN.compile(optimizer='adam', loss = 'mean_squared_error')

In [41]:
regressor_RNN.fit(X_train, y_train, epochs=256, batch_size=64, validation_data = (X_validation, y_validation),callbacks=[checkpoint_RNN, early_stopping])

Train on 5734 samples, validate on 442 samples
Epoch 1/256
5504/5734 [===========================>..] - ETA: 0s - loss: 0.0030
Epoch 00001: val_loss improved from inf to 0.00360, saving model to best_params_RNN.hdf5
5734/5734 [==============================] - 2s 378us/sample - loss: 0.0029 - val_loss: 0.0036
Epoch 2/256
5696/5734 [============================>.] - ETA: 0s - loss: 4.1443e-04
Epoch 00002: val_loss improved from 0.00360 to 0.00258, saving model to best_params_RNN.hdf5
5734/5734 [==============================] - 1s 176us/sample - loss: 4.1499e-04 - val_loss: 0.0026
Epoch 3/256
5504/5734 [===========================>..] - ETA: 0s - loss: 2.3114e-04
Epoch 00003: val_loss improved from 0.00258 to 0.00209, saving model to best_params_RNN.hdf5
5734/5734 [==============================] - 1s 173us/sample - loss: 2.2859e-04 - val_loss: 0.0021
Epoch 4/256
5504/5734 [===========================>..] - ETA: 0s - loss: 1.6974e-04
Epoch 00004: val_loss did not improve from 0.00209
57

5504/5734 [===========================>..] - ETA: 0s - loss: 2.3544e-05
Epoch 00032: val_loss did not improve from 0.00048
5734/5734 [==============================] - 1s 174us/sample - loss: 2.3050e-05 - val_loss: 4.9556e-04
Epoch 33/256
5504/5734 [===========================>..] - ETA: 0s - loss: 1.2046e-05
Epoch 00033: val_loss did not improve from 0.00048
5734/5734 [==============================] - 1s 176us/sample - loss: 1.1955e-05 - val_loss: 5.5278e-04
Epoch 34/256
5568/5734 [============================>.] - ETA: 0s - loss: 2.0395e-05
Epoch 00034: val_loss did not improve from 0.00048
5734/5734 [==============================] - 1s 171us/sample - loss: 2.0249e-05 - val_loss: 5.5794e-04
Epoch 35/256
5504/5734 [===========================>..] - ETA: 0s - loss: 1.7846e-05
Epoch 00035: val_loss did not improve from 0.00048
5734/5734 [==============================] - 1s 174us/sample - loss: 1.7711e-05 - val_loss: 5.1930e-04
Epoch 36/256
5504/5734 [===========================>..] -

## LSTM 

In [42]:
regressor_LSTM = Sequential()
regressor_LSTM.add(LSTM(units = 32, activation = 'tanh', input_shape = (X_train.shape[1], X_train.shape[2])))
regressor_LSTM.add(Dense(units = 1))

In [43]:
regressor_LSTM.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5504      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 5,537
Trainable params: 5,537
Non-trainable params: 0
_________________________________________________________________


In [44]:
checkpoint_LSTM = ModelCheckpoint(filepath="best_params_LSTM.hdf5", monitor="val_loss",verbose=1,save_best_only=True)

In [45]:
regressor_LSTM.compile(optimizer='adam', loss = 'mean_squared_error')

In [46]:
regressor_LSTM.fit(X_train, y_train, epochs=256, batch_size=64, validation_data = (X_validation, y_validation),callbacks=[checkpoint_LSTM, early_stopping])

Train on 5734 samples, validate on 442 samples
Epoch 1/256
5568/5734 [============================>.] - ETA: 0s - loss: 0.0546
Epoch 00001: val_loss improved from inf to 0.00826, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 5s 813us/sample - loss: 0.0531 - val_loss: 0.0083
Epoch 2/256
5568/5734 [============================>.] - ETA: 0s - loss: 6.8280e-04
Epoch 00002: val_loss improved from 0.00826 to 0.00495, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 2s 379us/sample - loss: 6.7270e-04 - val_loss: 0.0050
Epoch 3/256
5696/5734 [============================>.] - ETA: 0s - loss: 2.4794e-04
Epoch 00003: val_loss improved from 0.00495 to 0.00401, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 2s 402us/sample - loss: 2.4753e-04 - val_loss: 0.0040
Epoch 4/256
5568/5734 [============================>.] - ETA: 0s - loss: 1.5476e-04
Epoch 00004: val_loss improved from 0.00401 to 0.0

5734/5734 [==============================] - 3s 492us/sample - loss: 3.5542e-05 - val_loss: 0.0017
Epoch 31/256
5696/5734 [============================>.] - ETA: 0s - loss: 3.3996e-05
Epoch 00031: val_loss improved from 0.00169 to 0.00162, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 3s 484us/sample - loss: 3.3971e-05 - val_loss: 0.0016
Epoch 32/256
5696/5734 [============================>.] - ETA: 0s - loss: 3.4232e-05-
Epoch 00032: val_loss improved from 0.00162 to 0.00155, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 3s 477us/sample - loss: 3.4231e-05 - val_loss: 0.0016
Epoch 33/256
5696/5734 [============================>.] - ETA: 0s - loss: 3.7552e-05- ETA: 0s - loss: 3.81
Epoch 00033: val_loss did not improve from 0.00155
5734/5734 [==============================] - 3s 466us/sample - loss: 3.7467e-05 - val_loss: 0.0016
Epoch 34/256
5696/5734 [============================>.] - ETA: 0s - loss: 3.2161e-05
Ep

Epoch 61/256
5632/5734 [============================>.] - ETA: 0s - loss: 2.2453e-05
Epoch 00061: val_loss did not improve from 0.00088
5734/5734 [==============================] - 3s 481us/sample - loss: 2.2373e-05 - val_loss: 8.8242e-04
Epoch 62/256
5696/5734 [============================>.] - ETA: 0s - loss: 2.0372e-05
Epoch 00062: val_loss improved from 0.00088 to 0.00084, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 3s 474us/sample - loss: 2.0330e-05 - val_loss: 8.3622e-04
Epoch 63/256
5696/5734 [============================>.] - ETA: 0s - loss: 2.0375e-05
Epoch 00063: val_loss improved from 0.00084 to 0.00082, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 3s 466us/sample - loss: 2.0313e-05 - val_loss: 8.1569e-04
Epoch 64/256
5696/5734 [============================>.] - ETA: 0s - loss: 2.3357e-05
Epoch 00064: val_loss did not improve from 0.00082
5734/5734 [==============================] - 3s 460us/sample 

5696/5734 [============================>.] - ETA: 0s - loss: 1.0513e-05
Epoch 00091: val_loss improved from 0.00047 to 0.00046, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 3s 518us/sample - loss: 1.0531e-05 - val_loss: 4.5863e-04
Epoch 92/256
5696/5734 [============================>.] - ETA: 0s - loss: 1.4951e-05
Epoch 00092: val_loss improved from 0.00046 to 0.00045, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 3s 504us/sample - loss: 1.4905e-05 - val_loss: 4.5206e-04
Epoch 93/256
5696/5734 [============================>.] - ETA: 0s - loss: 1.1076e-05
Epoch 00093: val_loss did not improve from 0.00045
5734/5734 [==============================] - 3s 490us/sample - loss: 1.1055e-05 - val_loss: 4.6090e-04
Epoch 94/256
5696/5734 [============================>.] - ETA: 0s - loss: 9.4783e-06
Epoch 00094: val_loss improved from 0.00045 to 0.00043, saving model to best_params_LSTM.hdf5
5734/5734 [====================

Epoch 122/256
5632/5734 [============================>.] - ETA: 0s - loss: 4.5286e-06
Epoch 00122: val_loss did not improve from 0.00030
5734/5734 [==============================] - 3s 505us/sample - loss: 4.4761e-06 - val_loss: 3.0039e-04
Epoch 123/256
5696/5734 [============================>.] - ETA: 0s - loss: 5.1204e-06- ETA: 0s - loss: 5.0708e-0
Epoch 00123: val_loss did not improve from 0.00030
5734/5734 [==============================] - 3s 491us/sample - loss: 5.1050e-06 - val_loss: 3.0885e-04
Epoch 124/256
5696/5734 [============================>.] - ETA: 0s - loss: 4.5104e-06
Epoch 00124: val_loss did not improve from 0.00030
5734/5734 [==============================] - 3s 475us/sample - loss: 4.5004e-06 - val_loss: 2.9846e-04
Epoch 125/256
5696/5734 [============================>.] - ETA: 0s - loss: 4.1483e-06
Epoch 00125: val_loss improved from 0.00030 to 0.00028, saving model to best_params_LSTM.hdf5
5734/5734 [==============================] - 3s 480us/sample - loss: 4.13

## GRU

In [47]:
regressor_GRU = Sequential()
regressor_GRU.add(GRU(units = 32, activation = 'tanh', input_shape = (X_train.shape[1], X_train.shape[2])))
regressor_GRU.add(Dense(units = 1))

In [48]:
regressor_GRU.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 32)                4224      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 4,257
Trainable params: 4,257
Non-trainable params: 0
_________________________________________________________________


In [49]:
checkpoint_GRU = ModelCheckpoint(filepath="best_params_GRU.hdf5", monitor="val_loss",verbose=1,save_best_only=True)

In [50]:
regressor_GRU.compile(optimizer='adam', loss = 'mean_squared_error')

In [51]:
regressor_GRU.fit(X_train, y_train, epochs=256, batch_size=64, validation_data = (X_validation, y_validation),callbacks=[checkpoint_GRU, early_stopping])

Train on 5734 samples, validate on 442 samples
Epoch 1/256
5632/5734 [============================>.] - ETA: 0s - loss: 0.0039
Epoch 00001: val_loss improved from inf to 0.00154, saving model to best_params_GRU.hdf5
5734/5734 [==============================] - 5s 840us/sample - loss: 0.0039 - val_loss: 0.0015
Epoch 2/256
5632/5734 [============================>.] - ETA: 0s - loss: 9.8578e-05- ETA: 
Epoch 00002: val_loss improved from 0.00154 to 0.00130, saving model to best_params_GRU.hdf5
5734/5734 [==============================] - 3s 463us/sample - loss: 9.8087e-05 - val_loss: 0.0013
Epoch 3/256
5696/5734 [============================>.] - ETA: 0s - loss: 5.6637e-05
Epoch 00003: val_loss improved from 0.00130 to 0.00126, saving model to best_params_GRU.hdf5
5734/5734 [==============================] - 3s 484us/sample - loss: 5.6477e-05 - val_loss: 0.0013
Epoch 4/256
5632/5734 [============================>.] - ETA: 0s - loss: 4.4205e-05
Epoch 00004: val_loss improved from 0.00126 to

5734/5734 [==============================] - 3s 496us/sample - loss: 5.0440e-06 - val_loss: 2.9898e-04
Epoch 31/256
5696/5734 [============================>.] - ETA: 0s - loss: 5.2494e-06
Epoch 00031: val_loss did not improve from 0.00030
5734/5734 [==============================] - 3s 473us/sample - loss: 5.2339e-06 - val_loss: 3.2339e-04
Epoch 32/256
5696/5734 [============================>.] - ETA: 0s - loss: 4.9969e-06- ETA: 0s - loss: 5.
Epoch 00032: val_loss did not improve from 0.00030
5734/5734 [==============================] - 3s 471us/sample - loss: 4.9951e-06 - val_loss: 3.3942e-04
Epoch 33/256
5696/5734 [============================>.] - ETA: 0s - loss: 5.2474e-06
Epoch 00033: val_loss improved from 0.00030 to 0.00029, saving model to best_params_GRU.hdf5
5734/5734 [==============================] - 3s 478us/sample - loss: 5.2599e-06 - val_loss: 2.8660e-04
Epoch 34/256
5696/5734 [============================>.] - ETA: 0s - loss: 4.3217e-06- ETA: 1s - loss: 4.3478e-0 - ET
E